In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import warnings
from sklearn.datasets import fetch_20newsgroups
import nltk
!pip install progressbar2
from progressbar import ProgressBar
bar = ProgressBar()
nltk.download('stopwords')
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('english')

#Import Data
dataset =pd.read_csv("/content/drive/MyDrive/clean_data.csv")
data = dataset['0']
print(data[0])

In [157]:
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer

In [158]:
import nltk
nltk.download('wordnet')
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [159]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stop_word_list and len(token) > 3:
            result.append(lemmatize_stemming(token))
     
    return result

def clean_data(data):
  data = data.replace('\n',' ')
  data = data.replace('\t',' ')
  data = data.map(lambda x: re.sub('[,\.\'!?();:$%&#"]', '', x))
  data = data.replace('\n','', regex=True)
  data = data.replace('\'','', regex=True)
  data = data.replace('-','', regex=True)
  data = data.replace('’','', regex=True)
  
  return data

In [160]:
import string
converted_list = []
data.astype(str)
data = data.astype(str) 
data=clean_data(data)

for element in data:
    element = element.translate(str.maketrans('','',string.punctuation))
    converted_list.append(element.strip())

processed_docs = []

for doc in bar(converted_list):
    processed_docs.append(preprocess(doc))

100% (49999 of 49999) |##################| Elapsed Time: 0:08:06 Time:  0:08:06


In [161]:
bigram = gensim.models.Phrases(processed_docs, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[processed_docs], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[processed_docs[0]]])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in bar(texts):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['content', 'patriot', 'peter_berg', 'thriller', 'recreat', 'boston_marathon_bomb', 'ensu', 'manhunt', 'follow', 'surpris', 'obliqu', 'moral', 'ambigu', 'movi', 'typic', 'straightforward', 'filmmak', 'patriot', 'take', 'unexpect', 'cynic', 'view', 'chao', 'rash', 'bureaucrat_infight', 'follow', 'bomb', 'question', 'whether', 'berg', 'intend', 'messag', 'grim', 'run', 'time', 'movi', 'celebr', 'grind', 'help', 'bring', 'bomber', 'justic', 'glimps', 'someth', 'complic', 'jingoism', 'realli', 'linger', 'scene', 'good', 'illustr', 'dichotomi', 'come', 'late', 'patriot', 'search', 'bomber_dzhokhar_tsarnaev', 'intensifi', 'brother', 'wife', 'katherin', 'melissa', 'benoist', 'bring', 'interrog', 'connect', 'case', 'lawyer', 'protest', 'right', 'steeli', 'interview', 'khandi', 'alexand', 'honey', 'shit', 'hollywood', 'line', 'intend', 'provok', 'cheer', 'audienc', 'despit', 'queasi', 'legal', 'underton', 'film', 'present', 'cheesi', 'line', 'question', 'nowher', 'interrog', 'exit', 'shrug', 'm

In [ ]:
import spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

data_words_trigrams = make_trigrams(processed_docs)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [163]:
dictionary = gensim.corpora.Dictionary(data_lemmatized)

In [164]:
count = 0                         #Testing if it is created
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 attack
1 audienc
2 avoid
3 block
4 bomb
5 bomber
6 bring
7 brother
8 bureaucrat_infight
9 case
10 celebr


In [165]:
'''
Remove very rare and very common words: (Probably not working for Turkish but I tried)
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [166]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in data_lemmatized]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

In [183]:

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 30, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [184]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.057*"investig" + 0.025*"document" + 0.024*"agent" + 0.020*"enforc" + 0.016*"prosecutor" + 0.015*"arrest" + 0.015*"file" + 0.012*"crime" + 0.012*"request" + 0.012*"agenc"


Topic: 1 
Words: 0.053*"video" + 0.035*"facebook" + 0.030*"twitter" + 0.027*"site" + 0.024*"network" + 0.024*"user" + 0.017*"platform" + 0.017*"social_medium" + 0.016*"internet" + 0.012*"post"


Topic: 2 
Words: 0.031*"worker" + 0.026*"cost" + 0.022*"rate" + 0.020*"program" + 0.013*"fund" + 0.011*"budget" + 0.010*"employ" + 0.009*"benefit" + 0.009*"poor" + 0.008*"wage"


Topic: 3 
Words: 0.007*"religi" + 0.007*"movement" + 0.007*"moral" + 0.007*"free" + 0.007*"civil" + 0.006*"speech" + 0.006*"fear" + 0.006*"progress" + 0.005*"freedom" + 0.005*"muslim"


Topic: 4 
Words: 0.029*"russian" + 0.022*"press" + 0.015*"journalist" + 0.015*"fire" + 0.014*"investig" + 0.013*"medium" + 0.013*"director" + 0.012*"comey" + 0.012*"spicer" + 0.009*"leak"


Topic: 5 
Words: 0.043*"market" + 0.037*"trade" + 0.020*"gr

In [192]:
from pandas import DataFrame
unseen_document = [""" India's states should declare an epidemic following a rise in deadly "black fungus" cases, the country's health authorities has said.

The normally rare infection, called mucormycosis, has a mortality rate of 50%, with some only saved by removing an eye or jaw bone.

But in recent months, India saw thousands of cases affecting recovered and recovering Covid-19 patients.

Doctors suspect there may be a link with the steroids used to treat Covid.

Diabetics are at particular risk, with doctors telling the BBC it seems to strike 12 to 15 days after recovery from Covid.

The deadly 'black fungus' maiming Covid patients
India's holiest river is swollen with bodies
Six hospitals, three days and a Covid nightmare
On Thursday, Health Ministry Joint Secretary Lav Agarwal wrote to India's 29 states to ask them to declare it an epidemic.

By doing so, the ministry will be able more closely to monitor what is happening in each state, and allow for better integration of treatment.

It is not clear exactly how many cases there have been across the country, which is currently in the grip of a deadly second Covid-19 wave which has left tens of thousands dead.  """]

df = DataFrame(unseen_document,columns=['text'])
docs = df['text']
docs=clean_data(docs)

def token(values):
    filtered_words = [word for word in values.split() if word not in stop_word_list]
    
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc
docs = docs.map(lambda x: token(x))
df = docs

bow_vector = dictionary.doc2bow(preprocess(df[0]))

In [193]:
# Data preprocessing step for the unseen document
count=0
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    count+=1
    if(count<6):
      print("Score: %{}\t Topic: {}".format(score*100, re. findall('"([^"]*)"', lda_model.print_topic(index, 10))))

Score: %37.302157282829285	 Topic: ['doctor', 'session', 'patient', 'drug', 'test', 'health', 'cancer', 'risk', 'treatment', 'hospit']
Score: %23.647862672805786	 Topic: ['black', 'studi', 'research', 'human', 'brain', 'racial', 'color', 'cell', 'race', 'male']
Score: %17.709460854530334	 Topic: ['refuge', 'syrian', 'german', 'soldier', 'forc', 'bomb', 'strike', 'region', 'fighter', 'terrorist']
Score: %9.814301878213882	 Topic: ['polic', 'murder', 'death', 'fire', 'dead', 'arrest', 'crime', 'protest', 'video', 'suspect']
Score: %4.485643655061722	 Topic: ['market', 'trade', 'growth', 'investor', 'global', 'economi', 'chine', 'rise', 'invest', 'stock']
